In [1]:
from dotenv import load_dotenv, find_dotenv
import warnings
warnings.simplefilter('ignore')

In [2]:
load_dotenv(find_dotenv('file.env'))

True

In [9]:
import openai,langchain
print("langchain->",langchain.__version__, "openai->",openai.__version__)

langchain-> 0.2.8 openai-> 1.35.14


# Role  Prompting

Role prompting involves asking the LLM to assume a specific role or identity before performing a given task, such as acting as a copywriter. This can help guide the model's response by providing a context or perspective for the task. To work with role prompts, you could iteratively:

1- Specify the role in your prompt, e.g., "As a copywriter, create some attention-grabbing taglines for AWS services."

2 - Use the prompt to generate an output from an LLM.

3 - Analyze the generated response and, if necessary, refine the prompt for better results.

In [6]:
# In this example, the LLM is asked to act as a futuristic robot band 
# conductor and suggest a song title related to the given theme and year

from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

llm = OpenAI(model = "gpt-3.5-turbo-instruct", temperature=0)

template = """"
            As a futuristic robot band conductor, I need you to help me come up with a song title.
            What's a cool song title for a song about {theme} in the year {year}?
"""

prompt = PromptTemplate(
    input_variables = ["theme", "year"],
    template = template)

input_data = {"theme":"interstellar travel","year":"3030"}

chain = LLMChain(llm = llm, prompt=prompt)

response = chain.invoke(input_data)

print("Theme: interstellar travel")
print("Year: 3030")
print("AI-generated song title:", response['text'])

Theme: interstellar travel
Year: 3030
AI-generated song title: 
"Galactic Odyssey: Journey to 3030"


This is a good prompt for several reasons:

- **Clear instructions:** The prompt is phrased as a clear request for help in generating a song title, and it specifies the context: "As a futuristic robot band conductor." This helps the LLM understand that the desired output should be a song title related to a futuristic scenario.

- **Specificity:** The prompt asks for a song title that relates to a specific theme and a specific year, "{theme} in the year {year}." This provides enough context for the LLM to generate a relevant and creative output. The prompt can be adapted for different themes and years by using input variables, making it versatile and reusable.

- **Open-ended creativity:** The prompt allows for open-ended creativity, as it doesn't limit the LLM to a particular format or style for the song title. The LLM can generate a diverse range of song titles based on the given theme and year.
Focus on the task: The prompt is focused solely on generating a song title, making it easier for the LLM to provide a suitable output without getting sidetracked by unrelated topics.

These elements help the LLM understand the user's intention and generate a suitable response.

# Few Shot Prompting

In [21]:
# Few Shot Prompting In the next example, the LLM is asked to provide the emotion 
# associated with a given color based on a few examples of color-emotion pairs:
from langchain import FewShotPromptTemplate
examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

example_formatter_template = """
Color: {color}
Emotion: {emotion}\n
"""

example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix = "Here are some examples of colors and the emotions associated with them:\n\n",
    suffix="\n\nNow, given a new color, identify the emotion associated with it:\n\nColor: {input}\nEmotion:",
    input_variables=  ["input"],
    example_separator = "\n")

formatted_prompt = few_shot_prompt.format(input="purple")

chain = LLMChain(llm = llm, prompt=PromptTemplate(template=formatted_prompt))

response  = chain.invoke({})

print("Colour","purple")
print("Emotion",response['text'])

Colour purple
Emotion  royalty
